## 1. Building an "in-the-wild" texture model

#### Prerequisites

A collection of "in-the-wild" images with 3D mesh fits.

The authors of:

> Zhu, Xiangyu, et al. "Face alignment across large poses: A 3d solution." CVPR 2016

Provide data [on their website](http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/main.htm) which can be used to bootstrap this. As an example, we show loading this data into the format we need to proceed, and then demonstrate building the texture model. The two files this example follows are:
```
300W-3D.zip
300W-3D-Face.zip
```
unzip them next to each other in a folder and replace `DATA_PATH` with the parent folder path. 

If you have your own collection of "in-the-wild" images, you can easily replace this first cell with your own loading code - as long as you load each image with a 3D `TriMesh` that menpo visualizes the 2D projection of correctly, you are good. A good sanity check is therefore:
```
%matplotlib inline
img.view()
PointCloud(fit_3d.points[:, :2]).view()
```
You should see that Menpo renders the points of the mesh on the face in the image.

# 1. Loading the data

Firstly we load the data we need in the right format. We need to produce:

- A list of tuples of `(feature_img, fit_trimesh_3d)`, where
  - `feature_img` has already had a dense image feature e.g. `fast_dist` extracted from it
  - All instances of `feature_img` are scaled such that the face occupies the same size in each image
  - `fit_trimesh_3d` lies directly on the dense feature image - i.e. Y, X are in units of pixels, and Z is of an arbitrary scale (so long as depth is meaningfully conveyed for z-buffering)
  - A good test for this last condition is to visualize the mesh using Menpo (see the example below)
  
In order to be memory efficient, we choose to construct a `LazyList` rather than a literal `list` here. This means each feature/fit pair is generated from disk on the fly when we access each element, keeping memory usage low.

In [ ]:
from functools import lru_cache
from pathlib import Path
import numpy as np

import scipy.io as sio
import menpo.io as mio

from menpo.base import LazyList
from menpo.feature import fast_dsift
from menpo.shape import PointCloud, TriMesh

from itwmm import (render_overlay_of_mesh_in_img, 
                   generate_texture_model_from_image_3d_fits)

# Replace DATA_PATH with the path to your data. It should have subdirectories:
#  300W-3D/
#  300W-3D-Face/
DATA_PATH = Path('~/Dropbox/itwmm_src_data/').expanduser()

# The diagonal range of the projected fit
DIAGONAL_RANGE = 180

# The image feature we wish to employ
FEATURE_F = fast_dsift

In [ ]:
@lru_cache(1)
def load_trilist():
    trilist = sio.loadmat(str(DATA_PATH / '300W-3D' / 'Code' / 'ModelGeneration' / 'model_info.mat'))['tri']
    # flip triangle faces and matlab -> Python indexing
    return trilist[[1, 0, 2]].T - 1     


def convert_raw_3d_fit_to_img_trimesh_fit(raw_3d_fit, img):
    img_points_2d = raw_3d_fit[:2][::-1].copy() - 1  # matlab 1 indexing
    img_points_2d[0] = img.shape[0] - img_points_2d[0]
    # Depth needs to be flipped
    img_points_3d = np.vstack([img_points_2d, raw_3d_fit[2] * -1]).T
    img_trimesh_3d = TriMesh(img_points_3d, trilist=load_trilist())
    return img_trimesh_3d


def load_img_and_fit(img_path):
    fit_rel_path = img_path.relative_to(DATA_PATH / '300W-3D').with_suffix('.mat')
    fit_path = DATA_PATH / '300W-3D-Face' / fit_rel_path

    img = mio.import_image(img_path).as_greyscale()
    raw_3d_fit = sio.loadmat(str(fit_path))['Fitted_Face']
    fit_trimesh_3d = convert_raw_3d_fit_to_img_trimesh_fit(raw_3d_fit, img)
    return img, fit_trimesh_3d


def load_data_with_feature_sample(img_path, err_proportion=0.0001):
    img, fit_trimesh_3d = load_img_and_fit(img_path)
    
    # rescale the image to the diagonal range we are using
    img.landmarks['fit_2d'] = PointCloud(fit_trimesh_3d.points[:, :2])
    img, tr = img.rescale_landmarks_to_diagonal_range(DIAGONAL_RANGE, return_transform=True)
    # note that we use the rescaled landmarks here
    fit_trimesh_3d.points[:, :2] = img.landmarks['fit_2d'].points
    fit_trimesh_3d.points[:, 2] = fit_trimesh_3d.points[:, 2] / tr.scale[0]
    # take the feature on the rescaled image
    feat = FEATURE_F(img)
    return img, feat, fit_trimesh_3d

In [ ]:
img_paths = list(mio.image_paths(DATA_PATH / '300W-3D' / '**/*'))
print('{} images with fits to read.'.format(len(img_paths)))

# we can make a LazyList which returns image/feature/fit triples when
# we iterate. This saves us loading everything into memory at once.
imgs_features_and_fits = LazyList.init_from_iterable(img_paths).map(
    load_data_with_feature_sample)

In [ ]:
%matplotlib inline
# Let's test one.
img, feat, fit_trimesh_3d = imgs_features_and_fits[100]

# Visual check that everything looks sensible
img.view()
feat.view(channels=0, new_figure=True)
render_overlay_of_mesh_in_img(fit_trimesh_3d, img).view(new_figure=True)

In [ ]:
# the routine to build the ITW texture model only requires 
# feat/fit pairs, so we get rid of the redundent images
features_and_fits = imgs_features_and_fits.map(lambda x: x[1:])

In [ ]:
# Confirm we get the format we need
print(features_and_fits[0])

# Building the ITW texture model

Now we are able to build the texture model. Given data in the right format, this is just a single function call.

Note however that this process is quite memory intensive. We've done all we can in loading the data lazily, we still need to constract a large data matrix X and a mask matrix, and solve RPCA which has some memory overhead.

As a rough guide, with the recommended settings here, around 1.8GB of RAM will be required for every 100 images used.

Feel free to tweak `n_imgs_for_rpca` and choose wheather to use the original 64-bit feature images or 32-bit conversions (which will half RAM requirements).

Also a deprecation warning will appear in red, but it's only a notice of a future deprecation - this will be addressed in a future release of `menpo3d`.

In [ ]:
def as_float32(img):
    img = img.copy()
    img.pixels = img.pixels.astype(np.float32)
    return img

features_and_fits_32bit = features_and_fits.map(lambda x: (as_float32(x[0]), x[1]))
n_imgs_for_rpca = 100

itw_texture_model, X, m = generate_texture_model_from_image_3d_fits(features_and_fits_32bit[:n_imgs_for_rpca])

In [ ]:
print(itw_texture_model)

In [ ]:
print('The matrices used for R-PCA were of shape: {} & {}'.format(X.shape, m.shape))

In [ ]:
%matplotlib qt
from itwmm.base import as_colouredtrimesh
# As a sanity check, visualize the masks found on the original geometry
# with the texture value extracted (mask = red)
i = 4
img, feat, fit_trimesh_3d = imgs_features_and_fits[i]
img_sampled = np.repeat(img.sample(PointCloud(fit_trimesh_3d.points[:, :2])), 3, axis=0).T
mask = m[i].reshape([fit_trimesh_3d.n_points, -1])[:, 0]
img_sampled[~mask, 0] = 1.
as_colouredtrimesh(fit_trimesh_3d, colours=img_sampled).view()

# 3. Exporting a ITW texture model

To use this model we just need to save it out and load it in to the fitting notebooks. Note that we need to save out some metadata along with the raw PCA basis to be able to use this again in the future. In particular, we need:

- The rescaling diagonal. We will need this at fit time to ensure the features are meaningful.
- The feature extraction function. Again we need to call this at fit time to produce a meaningful feature image to compare against in our LK algorithm.

We also need the shape model we use to be in perfect correspondence with the fits used here.

In [ ]:
mio.export_pickle(
    {
        'texture_model': itw_texture_model,
        'diagonal_range': 180,
        'feature_function': fast_dsift
    }, 
    DATA_PATH / 'itw_texture_model.pkl',
    protocol=4, overwrite=True)